In [21]:
from owlready2 import *
import json

onto = get_ontology("appendix_ontology.owl")

onto.load()

ontology_entity = {}

for subclass in Thing.subclasses():
	print(subclass.name, type(subclass.name))
	subclass_name_str = subclass.name.replace("_", " ")
	if subclass_name_str not in ontology_entity:
		ontology_entity[subclass_name_str] = []
	for subsubclass in subclass.subclasses():
		subsubclass_name_str = subsubclass.name.replace("_", " ")
		ontology_entity[subclass_name_str].append(subsubclass_name_str)

with open("ontology_entity.json", "w", encoding="utf-8") as f:
	json.dump(ontology_entity, f, ensure_ascii=False, indent=4)

ngành_nghề_kinh_doanh_có_điều_kiện <class 'str'>


In [27]:
import json
from owlready2 import *
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

onto = get_ontology("appendix_ontology.owl")

onto.load()

ontology_entity = []

for subclass in Thing.subclasses():
	for subsubclass in subclass.subclasses():
		subsubclass_name_str = subsubclass.name.replace("_", " ")
		ontology_entity.append(subsubclass_name_str)

with open("dieu_document_only.json", "r", encoding="utf-8") as f:
	dieu_document = json.load(f)

def preprocess_text(text):
	text = text.lower()
	return text

def find_superclass(class_name):
	for subclass in Thing.subclasses():
		for subsubclass in subclass.subclasses():
			if subsubclass.name == class_name:
				return subclass.name

all_texts = ontology_entity + list(dieu_document.values())
preprocess_texts = [preprocess_text(text) for text in all_texts]

vectorizer = TfidfVectorizer()
tf_idf_matrix = vectorizer.fit_transform(preprocess_texts)

similarity_matrix = cosine_similarity(tf_idf_matrix[:len(ontology_entity)], tf_idf_matrix[len(ontology_entity):])

print("Similarity scores:")
# for i, entity in enumerate(ontology_entity["ngành nghề kinh doanh có điều kiện"]):
# 	highest_similarity_score = -1
# 	for j, doc_key in enumerate(dieu_document):
# 		# Get the doc_key with the highest similarity score when comparing with the entity
# 		if similarity_matrix[i][j] > highest_similarity_score:
# 			highest_similarity_score = similarity_matrix[i][j]
# 			highest_similarity_doc_key = doc_key
# 	if highest_similarity_score > 0.5:
# 		print(f"{entity} - {highest_similarity_doc_key}: {highest_similarity_score}")

# for i, entity in enumerate(ontology_entity["ngành nghề kinh doanh có điều kiện"]):
#     # Initialize a list to store (similarity_score, doc_key) tuples
#     similarity_scores = []
#     for j, doc_key in enumerate(dieu_document):
#         # Append the similarity score along with the corresponding doc_key
#         similarity_scores.append((similarity_matrix[i][j], doc_key))
    
#     # Sort the list by similarity score in descending order
#     similarity_scores.sort(reverse=True, key=lambda x: x[0])
    
#     # Select the top 5 entries
#     top_5_scores = similarity_scores[:1]
    
#     # Print the top 5 entries if the highest score is greater than 0.5
#     if top_5_scores[0][0] > 0.5:
#         print(f"{entity} - Top 5 Similar Documents:")
#         for score, doc_key in top_5_scores:
#             print(f"{doc_key}: {score}")
            
for j, doc_key in enumerate(dieu_document):
	# Initialize a list to store (similarity_score, entity) tuples
	similarity_scores = []
	for i, entity in enumerate(ontology_entity):
		# Append the similarity score along with the corresponding entity
		similarity_scores.append((similarity_matrix[i][j], entity))
	
	# Sort the list by similarity score in descending order
	similarity_scores.sort(reverse=True, key=lambda x: x[0])
	
	# Select the top 5 entries
	top_5_scores = similarity_scores[:1]
	
	# Print the top 5 entries if the highest score is greater than 0.5
	if top_5_scores[0][0] > 0.5:
		print("Checking: ")
		print(f" {doc_key} : {dieu_document[doc_key]} - Top 1 Similar Entities:")
		for score, entity in top_5_scores:
			print("Matching with:")
			print(f" {entity}: \nSimilarity Score: \n {score}")
			entity_text = entity.replace(" ", "_")
			# print(f"Superclass: {find_superclass(entity_text)}")
			dieu_document[doc_key] = (find_superclass(entity_text)).replace("_", " ")
			print(f"Equivalent to: \n {dieu_document[doc_key]}")
			print("\n\n\n\n\n\n")

with open("dieu_document_annotated.json", "w", encoding="utf-8") as f:
	json.dump(dieu_document, f, ensure_ascii=False, indent=4)

Similarity scores:


In [26]:
import json
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

with open("dieu_document_only.json", "r", encoding="utf-8") as f:
	dieu_document = json.load(f)

with open("86_2015_ND-CP_292146_only_dieu_jurisdict_agency_jurisdiction.json", "r", encoding="utf-8") as f:
	authority_data = json.load(f)

def preprocess_text(text):
	text = text.lower()
	return text

# authority_data["allow"] is a dictionary with each value being a list of strings
all_texts = list(dieu_document.values()) + [item for sublist in authority_data["allow"].values() for item in sublist] + [item for sublist in authority_data["not_allow"].values() for item in sublist]
preprocess_texts = [preprocess_text(text) for text in all_texts]

vectorizer = TfidfVectorizer()
tf_idf_matrix = vectorizer.fit_transform(preprocess_texts)

similarity_matrix = cosine_similarity(tf_idf_matrix[:len(dieu_document)], tf_idf_matrix[len(dieu_document):])

for j, doc_key in enumerate(dieu_document):
	# Initialize a list to store (similarity_score, authority) tuples
	similarity_scores = []
	for i, authority in enumerate([item for sublist in authority_data["allow"].values() for item in sublist] + [item for sublist in authority_data["not_allow"].values() for item in sublist]):
		# Append the similarity score along with the corresponding authority
		similarity_scores.append((similarity_matrix[j][i], authority))
	
	# Sort the list by similarity score in descending order
	similarity_scores.sort(reverse=True, key=lambda x: x[0])
	
	# Select the top 5 entries
	top_5_scores = similarity_scores[:1]
	
	# Print the top 5 entries if the highest score is greater than 0.5
	if top_5_scores[0][0] > 0.5:
		# print(f"Checking Article: {doc_key} \n {dieu_document[doc_key]}")
		print(f"Checking Article: {doc_key} by ủy ban nhân dân")
		print("Matching with:")
		for score, authority in top_5_scores:
			print(f"{authority}: \nSimilarity Score: {score}")
		print("Related Agency: hội đồng nhân dân")
		print("Authority: hasNoJurisdiction")
		print("Result: False")
		print("\n\n\n\n\n\n\n")

Checking Article: 1 by ủy ban nhân dân
Matching with:
 tại Khoản 1, Khoản 2 Điều này, Hội đồng nhân dân cấp tỉnh quy định mức học phí cụ thể hàng năm phù hợp với thực tế của các vùng trên địa bàn của mình. 1. Khung học phí đối với các chương trình giáo dục đại trà cấp học mầm non và giáo dục phổ thông công lập năm học 2015 - 2016 được quy định như sau: Điều 4. Khung học phí đối với giáo dục mầm non và phổ thông Đơn vị: 1.000 đồng/tháng/học sinh : 
Similarity Score: 0.5575226280416791
Related Agency: hội đồng nhân dân
Authority: hasNoJurisdiction
Result: False








